## Upload CSV Files

In [4]:
import pandas as pd
import sys
import os, os.path
from collections import defaultdict

data_dir = os.path.join(os.getcwd(), 'data/')

rep_and_district_info_filename = 'Names_Districts_Counties.csv'
county_asthma_info_filename = 'Asthma_Data_ALA_6.26.2019.csv'
county_polling_info_filename = 'Yale_Polling.csv'
voting_history_filename = 'vote_history.csv'
state_senate_folder = os.path.join(data_dir, 'daily-kos/state-senate-districts-to-counties/')
state_house_folder = os.path.join(data_dir, 'daily-kos/state-house-districts-to-counties/')

rep_and_district_info = pd.read_csv(data_dir + rep_and_district_info_filename, encoding = "ISO-8859-1")
county_asthma_info = pd.read_csv(data_dir + county_asthma_info_filename, encoding = "ISO-8859-1")
county_polling_info = pd.read_csv(data_dir + county_polling_info_filename, encoding = "ISO-8859-1")
voting_history = pd.read_csv(data_dir + voting_history_filename, encoding = "ISO-8859-1")

district_county_info = {}
for filename in os.listdir(state_senate_folder):
    state_abr = os.path.splitext(filename)[0]
    file_path = os.path.join(state_senate_folder, filename)
    district_county_info[state_abr] = {
        "SS": pd.read_csv(file_path, encoding = "ISO-8859-1")
    }
    
for filename in os.listdir(state_house_folder):
    state_abr = os.path.splitext(filename)[0]
    file_path = os.path.join(state_house_folder, filename)
    district_county_info[state_abr]["SH"] = pd.read_csv(file_path, encoding = "ISO-8859-1")


In [5]:
# Make sure the data was read correctly and is what we expect

print(rep_and_district_info.head(3))
print("-"*40)
print(county_asthma_info.head(3))
print("-"*40)
print(county_polling_info.head(3))
print("-"*40)
print(voting_history.head(3))
print("-"*40)
print(district_county_info["VA"]["SS"].head(3))
print("-"*40)
print(district_county_info["VA"]["SH"].head(3))

   Branch First Name Last Name District Party  \
0  Senate     George    Barker       39     D   
1  Senate    Richard     Black       13     R   
2  Senate   Jennifer    Boysko       33     D   

                                            Counties  
0  Fairfax County, Prince William County, Alexand...  
1            Loudoun County, Prince William County,   
2                   Fairfax County, Loudoun County,   
----------------------------------------
            County Total Pop Under 18 65 & Over Pediatric Asthma Adult Asthma  \
0         Accomack    32,545    6,756     7,402              536        2,245   
1        Albemarle   107,702   21,636    19,153            1,719        7,565   
2  Alexandria City   160,035   28,866    17,715            2,293       11,685   

    COPD Lung Cancer CV Disease Diabetes Poverty Estimate  
0  2,045          18      2,723    3,222            5,715  
1  6,096          59      7,726    9,287            8,015  
2  8,058          88      9,353   11,

## Generate District to County Info.


In [6]:
# rep_and_district_info_counties = rep_and_district_info['Counties']
# rep_and_district_info_districts = rep_and_district_info['District']

# #Clean up district values
# rep_and_district_info_counties = pd.Series(rep_and_district_info_counties.str.split(pat=","))

# all_counties = []
# county_lists = []
# for district_to_counties_list in rep_and_district_info_counties:
#     sanitized_counties = []
#     for county in district_to_counties_list:
#         sanitized_county_name = county.replace(' County','').strip()
#         if sanitized_county_name:
#             all_counties.append(sanitized_county_name)
#             sanitized_counties.append(sanitized_county_name)
#     county_lists.append(sanitized_counties)    
            
# districts = list(rep_and_district_info['District'])
# for i in range(len(districts)):        
#     if "th" in districts[i] or "st" in districts[i] or "nd" in districts[i] or "rd" in districts[i]:
#             districts[i] = districts[i][:-2]
            
# district_to_counties = dict(zip(districts, county_lists))

# all_counties = list(set(all_counties))
# all_counties = list([i for i in all_counties if i])

district_to_counties_by_state = {}
for state, state_info in district_county_info.items():
    district_to_counties = {}
    
    to_parse = [
        ("SS", "SD #.1", "County.1", "County Pop.\nin SD", "% of SD\nin County"),
        ("SH", "HD #.1", "County.3", "County Pop.\nin HD", "% of HD\nin County"),
    ]

    ss_info = state_info["SS"]
    
    for (chamber, district_num_key, county_num_key, county_pop_key, percent_key) in to_parse:
        for idx, row in state_info[chamber].iterrows():
            district_num = row.get(district_num_key)
            county = row.get(county_num_key)
            county_pop_in_district = row.get(county_pop_key)
            percent_of_district_in_county = row.get(percent_key)

            if not district_num or not county:
                continue

            district_num = district_num if type(district_num) == str else str(int(district_num))
            district_key = "%s-%s" % (chamber, district_num)

            if district_key not in district_to_counties:
                district_to_counties[district_key] = []

            district_to_counties[district_key].append((
                county.lower(), 
                county, 
                county_pop_in_district, 
                percent_of_district_in_county))
        
    district_to_counties_by_state[state] = district_to_counties   

In [7]:
# Sanity check the outputs

district_to_counties_by_state["VA"]

{'SS-1': [('james city', 'James City', 18847.0, 0.094),
  ('york', 'York', 9144.0, 0.045),
  ('hampton city', 'Hampton city', 16838.0, 0.084),
  ('newport news city', 'Newport News city', 137773.0, 0.685),
  ('suffolk city', 'Suffolk city', 4402.0, 0.022000000000000002),
  ('williamsburg city', 'Williamsburg city', 14068.0, 0.07)],
 'SS-2': [('york', 'York', 26867.0, 0.134),
  ('hampton city', 'Hampton city', 114610.0, 0.5720000000000001),
  ('newport news city', 'Newport News city', 42946.0, 0.215),
  ('portsmouth city', 'Portsmouth city', 15770.0, 0.079)],
 'SS-3': [('gloucester', 'Gloucester', 36858.0, 0.18100000000000002),
  ('isle of wight', 'Isle of Wight', 18951.0, 0.09300000000000001),
  ('james city', 'James City', 48162.0, 0.237),
  ('king and queen', 'King and Queen', 6945.0, 0.034),
  ('king william', 'King William', 15935.0, 0.078),
  ('new kent', 'New Kent', 18429.0, 0.091),
  ('york', 'York', 29453.0, 0.145),
  ('hampton city', 'Hampton city', 5988.0, 0.02899999999999999

In [8]:
# Drop labels 133 to get rid of "total" column
asthma_info_counties = [name.lower() for name in county_asthma_info['County'].drop(labels=133)]
asthma_info_children = list(county_asthma_info['Pediatric Asthma'].drop(labels=133))
asthma_info_adults = list(county_asthma_info['Adult Asthma'].drop(labels=133))

asthma_info_children = [int(x.replace(',', '')) for x in asthma_info_children]
asthma_info_adults = [int(x.replace(',', '')) for x in asthma_info_adults]

county_to_asthma_children = dict(zip(asthma_info_counties, asthma_info_children))
county_to_asthma_adults = dict(zip(asthma_info_counties, asthma_info_adults))

In [9]:
county_to_asthma_adults

{'accomack': 2245,
 'albemarle': 7565,
 'alexandria city': 11685,
 'alleghany': 1064,
 'amelia': 904,
 'amherst': 2209,
 'appomattox': 1076,
 'arlington': 17210,
 'augusta': 5311,
 'bath': 315,
 'bedford': 5461,
 'bland': 465,
 'botetourt': 2349,
 'bristol city': 1167,
 'brunswick': 1174,
 'buchanan': 1549,
 'buckingham': 1230,
 'buena vista city': 450,
 'campbell': 3871,
 'caroline': 2056,
 'carroll': 2104,
 'charles city': 516,
 'charlotte': 830,
 'charlottesville city': 3592,
 'chesapeake city': 16141,
 'chesterfield': 23149,
 'clarke': 1013,
 'colonial heights city': 1184,
 'covington city': 393,
 'craig': 360,
 'culpeper': 3391,
 'cumberland': 686,
 'danville city': 2799,
 'dickenson': 1032,
 'dinwiddie': 1987,
 'emporia city': 355,
 'essex': 775,
 'fairfax': 78010,
 'fairfax city': 1622,
 'falls church city': 968,
 'fauquier': 4695,
 'floyd': 1097,
 'fluvanna': 1851,
 'franklin': 3967,
 'franklin city': 544,
 'frederick': 5844,
 'fredericksburg city': 2000,
 'galax city': 452,
 '

# Asthma

## Sum childhood, adult, and total asthma for the counties that make up each district

Text: In the four counties that make up House District 100, 10,000 kids and 20,000 adults live with asthma.

In [10]:
# Make a mapping of istrict to asthma totals, # of counts of district.

district_to_asthma_text = {}
for district, county_info in district_to_counties_by_state["VA"].items():  
    chamber_abbr = district[:2]
    chamber = "State Senate" if chamber_abbr == "SS" else "State House" if chamber_abbr == "SH" else ""
    district_num = district[3:]
    child_asthma_count = 0
    adult_asthma_count = 0
    for (county_key, county, population, percent_of_district) in county_info:
        child_asthma_count += county_to_asthma_children[county_key]
        adult_asthma_count += county_to_asthma_adults[county_key]
    
    if len(county_info) > 1:
        district_to_asthma_text[district] =  """In the %d counties that make up %s District %s, %d kids and %d adults live with asthma.""" % (len(county_info), chamber, district_num, child_asthma_count, adult_asthma_count)
    else:
        district_to_asthma_text[district] =  """In the %s District %s, %d kids and %d adults live with asthma.""" % (chamber, district_num, child_asthma_count, adult_asthma_count)

In [11]:
district_to_asthma_text

{'SS-1': 'In the 6 counties that make up State Senate District 1, 9944 kids and 38595 adults live with asthma.',
 'SS-2': 'In the 4 counties that make up State Senate District 2, 8604 kids and 32555 adults live with asthma.',
 'SS-3': 'In the 10 counties that make up State Senate District 3, 8664 kids and 34346 adults live with asthma.',
 'SS-4': 'In the 10 counties that make up State Senate District 4, 6571 kids and 25005 adults live with asthma.',
 'SS-5': 'In the 2 counties that make up State Senate District 5, 8496 kids and 33607 adults live with asthma.',
 'SS-6': 'In the 5 counties that make up State Senate District 6, 12639 kids and 52192 adults live with asthma.',
 'SS-7': 'In the 2 counties that make up State Senate District 7, 11808 kids and 48489 adults live with asthma.',
 'SS-8': 'In the State Senate District 8, 7956 kids and 31023 adults live with asthma.',
 'SS-9': 'In the 4 counties that make up State Senate District 9, 11125 kids and 46682 adults live with asthma.',
 '

# Polling

## Create a weighted average of polling results for the counties that make up each district

Text: 

In the counties that make up this district, [weighted average]% know that global warming is happening, and [weighted average] are somewhat or very worried about it. [weighted average]% support regulating CO2 as a pollutant, and [weighted average] want to provide tax rebates for people who purchase energy-efficient vehicles or solar panels. 


- Number of people who know that global warming is happening: 
    - [[x%]] in [[least populous county]], and [[y%]] in [[most populous county]]
- Percent who are somewhat or very worried about climate change: 
    - [[x%]] in [[least populous county]], and [[y%]] in [[most populous county]]
- Support regulating CO2 as a pollutant: 
    - [[x%]] in [[least populous county]], and [[y%]] in [[most populous county]]
- Support tax rebates for people who purchase energy-efficient vehicles or solar panels: 
    - [[x%]] in [[least populous county]], and [[y%]] in [[most populous county]]


In [12]:
county_polling_info_counties = [name.lower() for name in county_polling_info['GeoName']]
# county_polling_info_pop = dict(zip(county_polling_info_counties,list(county_polling_info['TotalPop'])))
county_polling_info_happening = dict(zip(county_polling_info_counties,list(county_polling_info['happening'])))
county_polling_info_worried = dict(zip(county_polling_info_counties,list(county_polling_info['worried'])))
county_polling_info_regulate = dict(zip(county_polling_info_counties,list(county_polling_info['regulate'])))
county_polling_info_rebates = dict(zip(county_polling_info_counties,list(county_polling_info['rebates'])))

In [13]:
district_to_overview = {}
for district,county_info in district_to_counties_by_state["VA"].items():    
    total_pop, know_num, worried_num, regulate_num, rebates_num = 0.0,0.0,0.0,0.0,0.0
    
    for (county_key, county, population, percent_of_district) in county_info:
        total_pop += population
        know_num += county_polling_info_happening[county_key] * population
        worried_num += county_polling_info_worried[county_key] * population
        regulate_num += county_polling_info_regulate[county_key] * population
        rebates_num += county_polling_info_rebates[county_key] * population
        
    district_to_overview[district] =  ("In the counties that make up"
                                       " this district, {0:.2f}% know that"
                                       " global warming is happening,"
                                       " and {1:.2f}% are somewhat or very"
                                       " worried about it. {2:.2f}% support"
                                       " regulating CO2 as a pollutant,"
                                       " and {3:.2f}% want to provide tax"
                                       " rebates for people who purchase"
                                       " energy-efficient vehicles or "
                                       "solar panels.").format(know_num/total_pop, worried_num/total_pop, regulate_num/total_pop, rebates_num/total_pop)

In [14]:
print(district_to_overview)

{'SS-1': 'In the counties that make up this district, 75.49% know that global warming is happening, and 64.28% are somewhat or very worried about it. 79.69% support regulating CO2 as a pollutant, and 83.36% want to provide tax rebates for people who purchase energy-efficient vehicles or solar panels.', 'SS-2': 'In the counties that make up this district, 74.35% know that global warming is happening, and 62.91% are somewhat or very worried about it. 79.57% support regulating CO2 as a pollutant, and 83.40% want to provide tax rebates for people who purchase energy-efficient vehicles or solar panels.', 'SS-3': 'In the counties that make up this district, 65.72% know that global warming is happening, and 54.34% are somewhat or very worried about it. 75.04% support regulating CO2 as a pollutant, and 80.47% want to provide tax rebates for people who purchase energy-efficient vehicles or solar panels.', 'SS-4': 'In the counties that make up this district, 66.73% know that global warming is ha

In [15]:
district_to_specific = {}
for district, county_info in district_to_counties_by_state["VA"].items():
    
    # Format: (county_key, county, population, percent_of_district)
    max_pop_county = max(county_info, key=lambda x:x[2]) # by population
    min_pop_county = min(county_info, key=lambda x:x[2]) # by population
    
    if not max_pop_county or not min_pop_county:
        continue
    min_pop_happening = county_polling_info_happening[min_pop_county[0]]
    min_pop_worried = county_polling_info_worried[min_pop_county[0]]
    min_pop_regulate = county_polling_info_regulate[min_pop_county[0]]
    min_pop_rebates = county_polling_info_rebates[min_pop_county[0]]
    max_pop_happening = county_polling_info_happening[max_pop_county[0]]
    max_pop_worried = county_polling_info_worried[max_pop_county[0]]
    max_pop_regulate = county_polling_info_regulate[max_pop_county[0]]
    max_pop_rebates = county_polling_info_rebates[max_pop_county[0]]
    
    district_to_specific[district] = ("- Number of people who know that global warming is happening:" +
        "\n\t- {0:.2f}% in {1}, and {2:.2f}% in {3}").format(min_pop_happening, min_pop_county[1], max_pop_happening, max_pop_county[1])
    district_to_specific[district] += (
    "\n- Percent who are somewhat or very worried about climate change: "
        "\n\t- {0:.2f}% in {1}, and {2:.2f}% in {3}").format(min_pop_worried, min_pop_county[1], max_pop_worried, max_pop_county[1])

    district_to_specific[district] += (
    "\n- Support regulating CO2 as a pollutant: "
        "\n\t- {0:.2f}% in {1}, and {2:.2f}% in {3}"
    ).format(min_pop_regulate, min_pop_county[1], max_pop_regulate, max_pop_county[1])

    district_to_specific[district] += (
    "\n- Support tax rebates for people who purchase energy-efficient vehicles or solar panels: "
        "\n\t- {0:.2f}% in {1}, and {2:.2f}% in {3}").format(min_pop_rebates, min_pop_county, max_pop_rebates, max_pop_county)

In [16]:
print(district_to_specific)

{'SS-1': "- Number of people who know that global warming is happening:\n\t- 70.08% in Suffolk city, and 76.53% in Newport News city\n- Percent who are somewhat or very worried about climate change: \n\t- 59.43% in Suffolk city, and 65.33% in Newport News city\n- Support regulating CO2 as a pollutant: \n\t- 78.83% in Suffolk city, and 80.18% in Newport News city\n- Support tax rebates for people who purchase energy-efficient vehicles or solar panels: \n\t- 82.63% in ('suffolk city', 'Suffolk city', 4402.0, 0.022000000000000002), and 83.48% in ('newport news city', 'Newport News city', 137773.0, 0.685)", 'SS-2': "- Number of people who know that global warming is happening:\n\t- 73.51% in Portsmouth city, and 75.49% in Hampton city\n- Percent who are somewhat or very worried about climate change: \n\t- 63.68% in Portsmouth city, and 63.69% in Hampton city\n- Support regulating CO2 as a pollutant: \n\t- 79.98% in Portsmouth city, and 80.42% in Hampton city\n- Support tax rebates for peop

# Opposition Research

1. Build out an excel file with the following:
   - Name of incumbent
   - Important bills, and a vote (positive/negative)
   - 2 descriptions for each bill: positive and negative
   - The correct description to use, based on the vote history

2. Match incumbent names into the correct description, then add all descriptions together to create an opposition research paragraph that looks like this:

When it comes to common sense clean energy reforms, [[incumbent]] has a mixed record. They voted to block Virginia from entering regional collaborative efforts to combat climate change, specifically by voting to prevent Virginia from entering the Regional Greenhouse Gas Initiative. States already in this program have seen lower electricity bills and less pollution – joining it is a no-brainer. [[He/She]] also voted against integrating environmental education into Virginia's classrooms. 

However, [[incumbent]]’s record is not all bad. [[incumbent last name]] voted to ensure that the State Corporation Commission cannot reject or cut climate-forward legislation without reason. They also voted to establish a Clean Energy Advisory Board, which will run a pilot program to allow low-to-moderate income households to receive rebates for solar panels. Finally, [[Incumbent]] voted to ensure that coal plants dispose of coal ash– one of the largest types of industrial waste generated in the United States– safely and sustainably.



In [17]:
rep_and_district_info_districts = list(rep_and_district_info['District'])
rep_and_district_info_chambers = list(rep_and_district_info['Branch'])
rep_and_district_info_candidate_first = list(rep_and_district_info['First Name'])
rep_and_district_info_candidate_last = list(rep_and_district_info['Last Name'])

sanitized_candidate_names = []
for first, last in zip(rep_and_district_info_candidate_first,rep_and_district_info_candidate_last):
    sanitized_candidate_names.append(first + " " + last)
    
candidate_to_district = {}
for (candidate, district, chamber) in zip(
        sanitized_candidate_names, 
        rep_and_district_info_districts,
        rep_and_district_info_chambers):
    district_num = -1
    if "th" in district or "st" in district or "nd" in district or "rd" in district:
         district_num = district[:-2]
    else:
         district_num = district
    chamber_abbr = "SS" if chamber == "Senate" else "SH" if chamber == "House" else "??"
    candidate_to_district[candidate] = "%s-%s" % (chamber_abbr, district_num)

In [18]:
candidate_to_voting_record_sentences = defaultdict(list)

for row in voting_history.values:
    candidate_name = row[1] + " " + row[2]
    for i in range(4, len(row)):
        candidate_to_voting_record_sentences[candidate_name].append(row[i].replace("Candidate", candidate_name))            

In [19]:
candidate_to_voting_record_sentences

defaultdict(list,
            {'George Barker': ['-',
              '-',
              '-',
              '-',
              'George Barker voted to make it more difficult for Virginia to enter regional collaborative efforts to combat climate change in the transportation sector (via the Transportation and Climate Initiative).',
              '-',
              '-',
              'George Barker voted to make it more difficult for Virginia to enter into a regional cap and trade program (RGGI). States in this program have seen lower electricity bills and more clean job growth - joining it makes common sense.',
              '-',
              '-',
              'George Barker voted to ensure that coal plants dispose of coal ash- one of the largest types of industrial waste generated in the United States-Â\xa0safely and sustainably.',
              '-',
              '-',
              "George Barker voted to improve monitoring of elecric utilities' energy efficienty programs, which promot

In [20]:
# Parse the voting record
    # We have positive and negative votes, but I think we really just need the sentences
    # Drop first x columns
candidate_to_voting_record_text = {}
for candidate,voting_record_sentences in candidate_to_voting_record_sentences.items():
    voting_record_text = ""
    for sentence in voting_record_sentences:
        if sentence != "-":
            voting_record_text += sentence + " "
    candidate_to_voting_record_text[candidate] = voting_record_text  

# Final Printing

In [21]:
# Write output to file
with open('output.txt', 'a') as outfile: 
    for candidate in sanitized_candidate_names:
        d = candidate_to_district[candidate]
        if d == 'Senate-39':
            print(d, candidate, district_to_specific.keys())
        district_text = (
            district_to_asthma_text[d] + "\n\n" + 
            district_to_overview[d] + "\n\n" + 
            district_to_specific[d] + "\n\n" + 
            candidate_to_voting_record_text[candidate]
        )
        
        outfile.write("_" * 80 + "\n")
        outfile.write(district_text + "\n")
        outfile.write("_" * 80 + "\n")

# Print output for convenience
with open('output.txt', 'r') as outfile:
    for line in outfile:
        print(line)

________________________________________________________________________________________________

In the House District 39, 21376 kids and 78010 adults live with asthma.



In the counties that make up this district, 77.25% know that global warming is happening, and 69.92% are somewhat or very worried about it. 78.79% support regulating CO2 as a pollutant, and 84.10% want to provide tax rebates for people who purchase energy-efficient vehicles or solar panels.



- Number of people who know that global warming is happening:

	- 77.25% in Fairfax, and 77.25% in Fairfax

- Percent who are somewhat or very worried about climate change: 

	- 69.92% in Fairfax, and 69.92% in Fairfax

- Support regulating CO2 as a pollutant: 

	- 78.79% in Fairfax, and 78.79% in Fairfax

- Support tax rebates for people who purchase energy-efficient vehicles or solar panels: 

	- 84.10% in Fairfax, and 84.10% in Fairfax



George Barker voted to make it more difficult for Virginia to enter regional collabora

	- 64.27% in Amherst, and 60.90% in Bedford

- Percent who are somewhat or very worried about climate change: 

	- 52.87% in Amherst, and 49.13% in Bedford

- Support regulating CO2 as a pollutant: 

	- 73.96% in Amherst, and 72.21% in Bedford

- Support tax rebates for people who purchase energy-efficient vehicles or solar panels: 

	- 78.97% in Amherst, and 78.38% in Bedford



Stephen Newman voted to preserve Virginia's ability to enter regional collaborative efforts to combat climate change in the transportation sector (via the Transportation and Climate Initiative). Stephen Newman voted to preserve Virginia's ability to enter into a regional cap and trade program (RGGI). States in this program have seen lower electricity bills and more clean job growth - joining it makes common sense. Stephen Newman voted to ensure that coal plants dispose of coal ash- one of the largest types of industrial waste generated in the United States-Â safely and sustainably. Stephen Newman voted to cont


In the counties that make up this district, 76.08% know that global warming is happening, and 64.61% are somewhat or very worried about it. 80.29% support regulating CO2 as a pollutant, and 83.67% want to provide tax rebates for people who purchase energy-efficient vehicles or solar panels.



- Number of people who know that global warming is happening:

	- 75.49% in Hampton City, and 76.53% in Newport News City

- Percent who are somewhat or very worried about climate change: 

	- 63.69% in Hampton City, and 65.33% in Newport News City

- Support regulating CO2 as a pollutant: 

	- 80.42% in Hampton City, and 80.18% in Newport News City

- Support tax rebates for people who purchase energy-efficient vehicles or solar panels: 

	- 83.92% in Hampton City, and 83.48% in Newport News City



Cia Price voted for a progressive reform to move Virginia from fossil fuels to clean energy by 2035. Cia Price voted to keep Virginia in innovative regional partnerships that work to improve transpo

- Number of people who know that global warming is happening:

	- 77.25% in Fairfax, and 77.25% in Fairfax

- Percent who are somewhat or very worried about climate change: 

	- 69.92% in Fairfax, and 69.92% in Fairfax

- Support regulating CO2 as a pollutant: 

	- 78.79% in Fairfax, and 78.79% in Fairfax

- Support tax rebates for people who purchase energy-efficient vehicles or solar panels: 

	- 84.10% in Fairfax, and 84.10% in Fairfax



Benton Chafin voted to preserve Virginia's ability to enter regional collaborative efforts to combat climate change in the transportation sector (via the Transportation and Climate Initiative). Benton Chafin voted to preserve Virginia's ability to enter into a regional cap and trade program (RGGI). States in this program have seen lower electricity bills and more clean job growth - joining it makes common sense. Benton Chafin voted to ensure that coal plants dispose of coal ash- one of the largest types of industrial waste generated in the United S


- Support regulating CO2 as a pollutant: 

	- 80.45% in Albemarle, and 80.45% in Albemarle

- Support tax rebates for people who purchase energy-efficient vehicles or solar panels: 

	- 85.23% in Albemarle, and 85.23% in Albemarle



David Toscano voted for a progressive reform to move Virginia from fossil fuels to clean energy by 2035. David Toscano voted to keep Virginia in innovative regional partnerships that work to improve transportation and limit climate pollution through RGGI and TCI (Regional Greenhouse Gas Initiative and the Transportation and Climate Initiative). David Toscano voted to integrate basic environmental education into classrooms, a proposal that had both industry and environmental group support. David Toscano voted to make it more difficult for Virginia to enter regional collaborative efforts to combat climate change in the transportation sector (via the Transportation and Climate Initiative). David Toscano voted to ensure that the State Corporation Commission c

...

- Percent who are somewhat or very worried about climate change: 

	- 48.60% in Wythe, and 48.92% in Washington

- Support regulating CO2 as a pollutant: 

	- 72.22% in Wythe, and 72.47% in Washington

- Support tax rebates for people who purchase energy-efficient vehicles or solar panels: 

	- 77.23% in ('wythe', 'Wythe', 13163.0, 0.066), and 77.97% in ('washington', 'Washington', 54876.0, 0.27699999999999997)



Charles Carrico voted to preserve Virginia's ability to enter regional collaborative efforts to combat climate change in the transportation sector (via the Transportation and Climate Initiative). Charles Carrico voted to preserve Virginia's ability to enter into a regional cap and trade program (RGGI). States in this program have seen lower electricity bills and more clean job growth - joining it makes common sense. Charles Carrico voted to ensure that coal plants dispose of coal ash- one of the largest types of industrial waste generated in the United States-Â safely and sus


Chris Jones voted against moving Virginia to clean energy by 2035, which stunts Virginia's growth as a leader on renewables. Virginia trails its neighbors in clean energy jobs, and Chris Jones's vote ensures that other states will reap the rewards of clean energy, not Virginia.  Chris Jones voted against letting Virginia partner with its neighbors to reduce climate pollution from cars and power plants through RGGI and the TCI (Regional Greenhouse Gas Initiative and the Transportation and Climate Initiative). This directly affects Virginia's ability to attract clean energy jobs and innovate in the transportation sector. Chris Jones voted against integrating basic environmental education to Virginia's classrooms, a proposal that had both industry and environmental group support. Chris Jones voted to preserve Virginia's ability to enter regional collaborative efforts to combat climate change in the transportation sector (via the Transportation and Climate Initiative). Chris Jones voted t

- Support regulating CO2 as a pollutant: 

	- 72.21% in Bedford, and 74.39% in Roanoke

- Support tax rebates for people who purchase energy-efficient vehicles or solar panels: 

	- 78.38% in ('bedford', 'Bedford', 4865.0, 0.024), and 80.57% in ('roanoke', 'Roanoke', 62098.0, 0.311)



David Suetterlein voted to preserve Virginia's ability to enter regional collaborative efforts to combat climate change in the transportation sector (via the Transportation and Climate Initiative). David Suetterlein voted to preserve Virginia's ability to enter into a regional cap and trade program (RGGI). States in this program have seen lower electricity bills and more clean job growth - joining it makes common sense. David Suetterlein voted against ensuring coal plants dispose of coal ash safely and sustainably. This threatens public safety by permitting pollution of waterways, ground water, drinking water, and the air. David Suetterlein voted to deplete offshore habitat and promote fossil fuel depend


	- 59.09% in Craig, and 60.90% in Bedford

- Percent who are somewhat or very worried about climate change: 

	- 48.25% in Craig, and 49.13% in Bedford

- Support regulating CO2 as a pollutant: 

	- 71.86% in Craig, and 72.21% in Bedford

- Support tax rebates for people who purchase energy-efficient vehicles or solar panels: 

	- 77.52% in ('craig', 'Craig', 5190.0, 0.026000000000000002), and 78.38% in ('bedford', 'Bedford', 70033.0, 0.349)



Stephen Newman voted to preserve Virginia's ability to enter regional collaborative efforts to combat climate change in the transportation sector (via the Transportation and Climate Initiative). Stephen Newman voted to preserve Virginia's ability to enter into a regional cap and trade program (RGGI). States in this program have seen lower electricity bills and more clean job growth - joining it makes common sense. Stephen Newman voted to ensure that coal plants dispose of coal ash- one of the largest types of industrial waste generated in the U

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

